# 模块和元类

元类是一个大的主题，但简而言之，它们是创建类的蓝图。换句话说，类创建一个实例，元类根据创建实例时的需求自动更改类的行为。

## 预备知识-元类
在 Python 中基于类可以创建实例对象

In [ ]:
# 定义类
class Foo(object):
    def __init__(self, name):
        self.name = name

    def __new__(cls, *args, **kwargs):
        data = object.__new__(cls)
        return data


# 根据类创建对象
#   1. 执行类的new方法，创建对象（空对象）【构造方法】  {}
#   2. 执行类的init方法，初始化对象。【初始化方法】  {name: 'Gavin'}
obj = Foo('Gavin')

类是由元类创建的
类默认由Type创建，Type是一个元类

In [4]:
# 传统方式创建
class Foo(object):
    v1 = 123

    def func(self):
        return 666


# 非传统方式创建
# type(__name__, __base__, attributes)
# 组成：类名（字符串）、继承类（元组）、成员（字典）
Fa = type("Foo", (object,), {'v1': 123, 'func': lambda self: 666})

obj = Fa()
print(obj.v1)

123


自定义创建元类

In [7]:
class MyType(type):
    def __init__(self, *args, **kwargs):
        print('init')
        super().__init__(*args, **kwargs)

    def __new__(cls, *args, **kwargs):
        print('new')
        # 创建类
        new_cls = super().__new__(cls, *args, **kwargs)
        return new_cls

    def __call__(self, *args, **kwargs):
        # 1. 调用自己的new方法来创建一个对象
        empty_object = self.__new__(self)

        # 2. 调用自己的init方法去初始化
        self.__init__(empty_object, *args, **kwargs)
        return empty_object


class Foo(object, metaclass=MyType):
    def __init__(self, name):
        self.name = name


foo = Foo('Gavin')
print(foo)
print(foo.name)

# Foo 类由 MyType 创建，先执行元类的new方法，再执行元类的init方法
# 元类创建类与类创建实例一样

new
init
Gavin


类中的 \_\_call__ 方法什么时候被调用？
obj() 会执行 \_\_call__ 方法

单例模式

In [11]:
class MyType(type):
    def __init__(self, name, bases, attrs):
        super().__init__(name, bases, attrs)
        self.instance = None

    def __call__(self, *args, **kwargs):
        if not self.instance:
            self.instance = self.__new__(self)

        self.__init__(self.instance, *args, **kwargs)

        return self.instance


class Singleton(object, metaclass=MyType):
    pass


class Foo(Singleton):
    pass


v1 = Foo()
v2 = Foo()
print(v1)
print(v2)

## 使用 \_\_init__ 文件
使用  \_\_init__ 文件 可以将多个功能粘合在一起
- purchase:
    - \_\_init__.py
    - cart.py
    - payment.py

In [ ]:
# cart module
class Cart:
    def add_to_cart(self, cart, product):
        self.execute_query_to_add(cart, product)
        print("successfully added to cart")


# payment module
class Payment:
    def do_payment(self, user, amount):
        self.execute_payment_query(user, amount)
        print(f"Payment of ${amount} successfully done!")


# __init__.py
from . import Cart
from . import Payment

## 使用元类改变类的行为

防止直接创建对象

In [15]:
class NoClassInstance:
    def __call__(self, *args, **kwargs):
        raise TypeError('Can\'t instantiate directly')


class User(metaclass=NoClassInstance):
    @staticmethod
    def print_name(name):
        print(f'Name: {name}')


user = User()

TypeError: NoClassInstance() takes no arguments

使用 \_\_call__ 设计 API

In [1]:
class Calculation:
    def __init__(self, operation):
        self.operation = operation

    def __call__(self, fist_number, second_number):
        if isinstance(fist_number, int) and isinstance(second_number, int):
            return self.operation()
        raise ValueError("Provide numbers")

    def add(self, first, second):
        return first + second

    def multiply(self, first, second):
        return first * second


add = Calculation(add)
print(add(5, 4))

NameError: name 'add' is not defined